Get code from github

In [1]:
! git clone https://github.com/balezz/bert4classification.git

Cloning into 'bert4classification'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 67 (delta 31), reused 33 (delta 10), pack-reused 0
Unpacking objects: 100% (67/67), done.


In [2]:
%cd bert4classification

/content/bert4classification


Installing Packages

In [3]:
!pip install -r reqirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 75 kB 1.4 MB/s 
     |████████████████████████████████| 4.2 MB 24.6 MB/s 
     |████████████████████████████████| 84 kB 2.7 MB/s 
     |████████████████████████████████| 596 kB 47.4 MB/s 
     |████████████████████████████████| 6.6 MB 33.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Download data

In [4]:
from minio import Minio

BUCKET_NAME = 'findatalake'
S3_FILE_NAME = 'rutwits.zip'
TMP_FILE = 'tmp.zip'
MINIO_URL = 'datalake.website:9000'

In [5]:
client = Minio(MINIO_URL,
               access_key='tester-1',
               secret_key='testerpass',
               secure=False)

In [6]:
client.fget_object(BUCKET_NAME, S3_FILE_NAME, TMP_FILE)

In [7]:
! unzip -a {TMP_FILE}

Archive:  tmp.zip
  inflating: test.csv                [binary]
  inflating: train.csv               [binary]
  inflating: valid.csv               [binary]


And so one...

In [8]:
import pandas as pd

In [9]:
train_data = pd.read_csv('train.csv')
valid_data = pd.read_csv('valid.csv')
test_data  = pd.read_csv('test.csv')

In [10]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

Initialize BERT classifier

In [11]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=2
)

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/45.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Downloading:   0%|          | 0.00/235k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Prepare data and helpers for train and evlauation

In [12]:
classifier.preparation(
        X_train=list(train_data['text']),
        y_train=list(train_data['label']),
        X_valid=list(valid_data['text']),
        y_valid=list(valid_data['label'])
    )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Train loop

In [13]:
classifier.train()

Epoch 1/2


KeyboardInterrupt: ignored

Check test data

In [ ]:
texts = list(test_data['text'])
labels = list(test_data['label'])

predictions = [classifier.predict(t) for t in texts]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')